In [1]:
#!pip install sqlalchemy
import sqlalchemy
import pandas as pd

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [11]:
engine = sqlalchemy.create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con= engine.connect()
rs = con.execute('''SELECT f.name AS FacilityName,
                 CASE
	WHEN f.guestcost*slots >0
	THEN f.guestcost*slots
	WHEN f.membercost*slots >0
	THEN f.membercost*slots
	END AS cost
	
	From (select firstname, surname, b.slots, b.facid from Members as m inner join Bookings as b on b.memid= m.memid
          WHERE b.starttime LIKE '2012-09-14%' ) as Cost
    inner join Facilities AS f on f.facid = Cost.facid
	 
                 ''')
                 
df= pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df=df.groupby('FacilityName').sum()
df[df.cost < 1000].sort_values(by='cost', ascending=False)

,cost
FacilityName,
Tennis Court 1,525.0
Tennis Court 2,525.0
Massage Room 2,480.0
Squash Court,280.0
Badminton Court,232.5
Pool Table,105.0
Snooker Table,50.0
Table Tennis,40.0


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [9]:
rs = con.execute('''SELECT m1.surname, m1.firstname, m2.surname || ' ' || m2.firstname as RecommendedBy
                    FROM Members as m1
                    INNER JOIN Members as m2 
                    ON m1.recommendedby = m2.memid
                    ORDER BY m1.surname, m1.firstname
                 ''')
                 
df= pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df

,surname,firstname,RecommendedBy
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren
5,Coplin,Joan,Baker Timothy
6,Crumpet,Erica,Smith Tracy
7,Dare,Nancy,Joplette Janice
8,Genting,Matthew,Butters Gerald
9,Hunt,John,Purview Millicent


/* Q12: Find the facilities with their usage by member, but not guests */

In [69]:
rs = con.execute('''SELECT u.name as facility, u.surname || ' ' || u.firstname as MemberUsage
                    FROM (SELECT f.name as name, m.surname as surname, m.firstname as firstname 
                            FROM Members as m
                            INNER JOIN Bookings as b
                            ON b.memid = m.memid
                            
                            INNER JOIN Facilities as f
                            ON b.facid = f.facid
                            WHERE b.facid <> 0) as u
                   ''')
                 
df= pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df = df.groupby('facility')['MemberUsage'].count()
df

facility
Badminton Court    383
Massage Room 1     629
Massage Room 2     111
Pool Table         836
Snooker Table      444
Squash Court       440
Table Tennis       403
Tennis Court 2     389
Name: MemberUsage, dtype: int64

/* Q13: Find the facilities usage by month, but not guests */

In [74]:
rs = con.execute('''SELECT u.name as facility, u.surname || ' ' || u.firstname as MemberUsage, u.starttime
                    FROM (SELECT f.name as name, m.surname as surname, m.firstname as firstname, b.starttime  as starttime
                            FROM Members as m
                            INNER JOIN Bookings as b
                            ON b.memid = m.memid
                            
                            INNER JOIN Facilities as f
                            ON b.facid = f.facid
                            WHERE b.facid <> 0) as u
                   ''')
                 
df= pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df.starttime = pd.to_datetime(df.starttime)
df.starttime = df.starttime.dt.month
df.rename(columns={'starttime': 'month'}, inplace=True)
df.columns
df.groupby(['facility','month']).count()

MemberUsage
facility        month             
Badminton Court 7               56
                8              146
                9              181
Massage Room 1  7              123
                8              224
                9              282
Massage Room 2  7               12
                8               40
                9               59
Pool Table      7              110
                8              291
                9              435
Snooker Table   7               75
                8              159
                9              210
Squash Court    7               75
                8              170
                9              195
Table Tennis    7               51
                8              147
                9              205
Tennis Court 2  7               68
                8              149
                9              172